In [5]:
import pdfplumber
import re


with pdfplumber.open("data\CE67-04 Logo_Identification.pdf") as pdf:
    full_text = ""
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        if text:
            full_text += f"\n--- Page {i+1} ---\n"
            full_text += text

def clean_text(text):

    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(Page\s+\d+|บทที่\s+\d+)', '', text, flags=re.IGNORECASE)

    return text.strip()

def split_into_chunks(text, max_words=300, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words - overlap):
        chunk = " ".join(words[i:i + max_words])
        chunks.append(chunk)
    return chunks

chunks = split_into_chunks(clean_text(full_text))

<>:5: SyntaxWarning: invalid escape sequence '\C'
<>:5: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Suppa\AppData\Local\Temp\ipykernel_16256\3456565638.py:5: SyntaxWarning: invalid escape sequence '\C'
  with pdfplumber.open("data\CE67-04 Logo_Identification.pdf") as pdf:
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
Crop

In [ ]:
#connect to pg db
import os
from dotenv import load_dotenv
import psycopg2

load_dotenv()

conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

cur = conn.cursor()
# cur.execute("CREATE EXTENSION vector;")
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            content TEXT,
            embedding vector(1024)
            )
""")
conn.commit()
cur.close()
conn.close()

In [9]:
from sentence_transformers import SentenceTransformer

conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

cur = conn.cursor()

embedder = SentenceTransformer("BAAI/bge-m3")

def add_document(text):
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (content,  embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()

for text in chunks:
    documents = add_document(text)

cur.close()
conn.close()




In [ ]:
def query_postgresql(query_text,k=3):
    query_embedding = embedder.encode(query_text).tolist()

    conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
    )
    cur = conn.cursor()

    query_embedding_str = "["+", ".join(map(str , query_embedding)) +"]"

    sql_query = """
        SELECT content, embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
        LIMIT %s;
"""
    cur.execute(sql_query,(query_embedding_str,k))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

results = query_postgresql("สมาชิกมีใครบ้าง")
results

[('ASSIGNED ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์, ุส ิพชชา ัภทร, ุสรัศกิ์ด, ศุภ ิวชญ์

In [18]:
import ollama

def generate_response(query_text):
    retrieved_docs = query_postgresql(query_text)

    context = "\n".join(doc[0] for doc in retrieved_docs)

    prompt = f"Answer the questions based on the following context:\n{context}\n\nQuestion:{query_text}"

    response = ollama.chat(model="llama3.2",messages=[
        {"role": "system","content":"You are an assistant."},
        {"role": "user","content":prompt},
    ])

    return response["message"]["content"]

generate_response("โมเดลที่ดีที่สุดของการทำ Logo classification คือตัวไหน และมีค่าความแม่นยำเท่าไหร่")

'ตามข้อมูลในบทความ โมเดลที่ดีที่สุดสำหรับการทำ Logo Classification คือ MobileNetV3 โดยมีค่าความแม่นยำเป็น mAP50 อยู่ที่ 0.9633 และ F1 score อยู่ที่ 0.9524 เมื่อผ่านชุด Test'